# Camada GOLD

## Informações Gerais
  | Information | Details |
  |---------|------|
  |Table Name | gold.pedidos|
  |Origem | silver.pedidos|

## Histórico de Atualizações
|Data | Desenvolvido por | Motivo |
|:------:|--------|-------|
|mm/dd/aaaa|Data Eng.|details|

In [0]:
### Import of libs

from pyspark.sql.functions import current_date, current_timestamp, expr

In [0]:
### Nome do Esquema e Tabela no catálogo

database = 'gold'
tabela = 'venda_total_estabelecimentos'

In [0]:
# Funções

## Função que aplica comentário na tabela!

def adicionaComentariosTabela(a,b,c,d):
    spark.sql(f"COMMENT ON TABLE {a}.{b} IS 'C'")
    for key,value in d.items():
        sqlaux = f"ALTER TABLE {a}.{b} CHANGE COLUMN {key} COMMENT '{value}'"
        spark.sql(sqlaux)

In [0]:
df_pedidos = spark.sql(
f"""
SELECT
    CAST(id_estabelecimento AS INT),
    nome_estabalecimento,
    Produto AS Nome_produto,
    CAST(SUM(quantidade_vendida) AS INT) AS quantidade_total,
    CAST(SUM(Preco_Unitario) AS DECIMAL(20,2)) AS valor_total
FROM
    silver.pedidos
GROUP BY all
""")

In [0]:
## Incluir colunas de controle

df_pedidos = df_pedidos.withColumn("data_carga", current_date())
df_pedidos = df_pedidos.withColumn("data_hora_carga", expr("current_timestamp() - INTERVAL 3 HOURS"))


In [0]:
## Na camada GOLS é necessário a existência de Metadados, para maior compreensão das colunas. 
#Isso pode ser feito de forma MANUAL ou automática

# para automatico, ver a função AdicionaComentariosTabela

comentario_tabela = "esta tabela é uma tabela corporativa para XXX usos"

lista_comentario_colunas ={
'id_estabelecimento' : 'ID do Establ (FK)',
'quantidade_total': 'Quantidade vendida na operação',
'nome_estabalecimento': 'nome fantasia',
'Nome_produto': 'string',
'valor_total': 'soma faturamento'
}

print('Dados Gravados!')


Dados Gravados!


In [0]:
df_pedidos.write \
    .format('delta') \
    .mode('overwrite') \
    .clusterBy("id_estabelecimento") \
    .option('mergeSchema','true') \
    .option('overwriteSchema','true') \
    .saveAsTable(f'{database}.{tabela}')
adicionaComentariosTabela(database,tabela,comentario_tabela,lista_comentario_colunas)
print("Dados gravados com sucesso")

Dados gravados com sucesso


In [0]:
spark.sql(f"OPTIMIZE {database}.{tabela}")
print("Otimização Finalizada")

Otimização Finalizada


In [0]:
%sql
--1 
SELECT *
FROM gold.venda_total_estabelecimentos
ORDER BY quantidade_total DESC --Pra mim, n faz mto sentido comparar diferentes produtos por QUANTIDADE! Melhor faturamento!!!!
LIMIT 5;

id_estabelecimento,nome_estabalecimento,Nome_produto,quantidade_total,valor_total,data_carga,data_hora_carga
29,Lanchonete Top,Cerveja XYZ,1488,42.35,2024-12-29,2024-12-28T23:52:29.783Z
50,Bar do João,Suco Natural,1313,50.75,2024-12-29,2024-12-28T23:52:29.783Z
39,Bar da Esquina,Vinho ABC,1268,38.05,2024-12-29,2024-12-28T23:52:29.783Z
27,Oficina Mecânica,Cerveja XYZ,1247,38.26,2024-12-29,2024-12-28T23:52:29.783Z
41,Pizzaria Italiana,Cerveja XYZ,1068,27.40,2024-12-29,2024-12-28T23:52:29.783Z
